# Notebook 8 - Iroha 1 Blockchain Tools

[From Python Iroha Examples](https://github.com/hyperledger-iroha/iroha-python/tree/1.1.3/examples)


## Get Blocks

[Infinite Block Stream](examples/infinite-blocks-stream.py)

Interrupt execution to stop streaming.

In [1]:
from integration_helpers import get_blocks
# from iroha_helper import *

In [ ]:
get_blocks()

2025-02-06 23:23:48.438 | DEBUG    | integration_helpers:tracer:35 - 	Entering "get_blocks"


2025-02-06 23:23:59.206 | INFO     | integration_helpers:get_blocks:156 - The next block arrived: block_response {
  block {
    block_v1 {
      payload {
        transactions {
          payload {
            reduced_payload {
              commands {
                call_engine {
                  caller: "admin@test"
                  callee: "BD0274D8D8C2EE62AEB41678578DA174A4CE7BA1"
                  input: "bc53c0c4000000000000000000000000000000000000000000000000000000000000006000000000000000000000000000000000000000000000000000000000000000a000000000000000000000000000000000000000000000000000000000000000e0000000000000000000000000000000000000000000000000000000000000000e666F63757365645F62656176657200000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000047465737400000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000004033343136383364373065396530373564663336303332626261366561353465